In [3]:
"""Generate sh file that runs our method"""
import os
model_name = "meta-llama/Llama-3.1-8b"
# model_name = "meta-llama/Llama-3.1-70B"
# model_name = "meta-llama/Llama-3.1-70B-Instruct"
# model_name = 'google/gemma-3-4b-pt'
# model_name = 'Qwen/Qwen3-8B'
# model_name = 'Qwen/Qwen3-32B'
# model_name = 'mistralai/Mistral-7B-v0.3'
# model_name = 'mistralai/Mistral-7B-Instruct-v0.3'
# model_name = 'mistralai/Mixtral-8x7B-v0.1'
# model_name = 'mistralai/Mixtral-8x7B-Instruct-v0.1'
data_name = 'icl'
num_example = 50    # M; the number of prompts used in the task embedding construction
num_shot = 10       # N; the number of input-output pairs for each prompt, used in the task embedding construction
max_token = 20      # The maximum number of tokens to be generated during inference
eval_num_shot = 0   # The number of shots (or examples) in test prompts (0: inference with 0-shot prompts)
cur_mode = 'interv' # 'interv': intervention with site, 'clean': clean run without any intervention
batch_size = 1      # Batch size used in the optimization of soft head-selection parameters. We strongly recommend using 1; Some LLMs may cause unexpected results during inference with batch size larger than 1 (refer to https://discuss.huggingface.co/t/ask-for-help-output-inconsistency-when-using-llm-batch-inference-compared-to-single-input/146303). However, for Llama-3.1 and Qwen3, we observe that it is safe to use batch size larger than 1.
test_batch_size = 1 # Batch size used in the inference. Exact match score only supports batch size 1.
dataset_names_list = [["ag_news"]] # Dataset name (or task name) to be evaluated. 
lr = 2e-1           # Learning rate used in the optimization of soft head-selection parameters
epoch = 400         # The number of training iterations for optimizing soft head-selection parameters. 400 indicates 400 (train) prompts are used in the optimization.
exact_match = True  # If False, first token match score is used instead of exact match score.
data_path = './dataset'
exp_name = f"{cur_mode}_task-agnostic_epoch_{epoch}_lr_{lr}"
gpu_nums = [0]

directory_path = "./"
file_name = "eval_site.sh"

if cur_mode == "clean":
    exp_name = f"{cur_mode}_{eval_num_shot}_shot_accuracy"
block_cnt = 0
cnt = 0
with open(os.path.join(directory_path, file_name), "w") as f:
    f.write("#!/bin/bash"+"\n\n")
    f.write(f'echo "Running ours method with model: {model_name}"'+'\n\n')
    f.write(f'echo "Running code block {block_cnt}"' + '\n')
    for dataset_names in dataset_names_list:
        if len(dataset_names) == 1:
            exp_name = f"{cur_mode}_{dataset_names[0]}_epoch_{epoch}_lr_{lr}"
        else:
            pass
        if cnt != 0 and cnt % len(gpu_nums) == 0:
            block_cnt += 1
            f.write("\n"+"wait"+"\n\n")
            f.write(f'echo "Running code block {block_cnt}"' + '\n')
        if '70b' in model_name.lower():
            if exact_match:
                f.write(f'CUDA_VISIBLE_DEVICES={gpu_nums[cnt % len(gpu_nums)]},{gpu_nums[(cnt+1) % len(gpu_nums)]} python eval.py --exact_match --model_name {model_name} --data_name {data_name} --num_example {num_example} --num_shot {num_shot} --max_token {max_token} --eval_num_shot {eval_num_shot} --activation_path {f'./site/{model_name.split('/')[-1]}/mean_activations'} --alphas_path {f'./site/{model_name.split('/')[-1]}/alphas/alphas_{exp_name}.pt'} --result_folder {f'./site/{model_name.split('/')[-1]}/result'} --cur_mode {cur_mode} --experiment_name {exp_name} --batch_size {batch_size} --test_batch_size {test_batch_size} --data_path {data_path} --dataset_names {' '.join(dataset_names[i] for i in range(len(dataset_names)))} --lr {lr} --epoch {epoch}&'+'\n')    
            else:
                f.write(f'CUDA_VISIBLE_DEVICES={gpu_nums[cnt % len(gpu_nums)]},{gpu_nums[(cnt+1) % len(gpu_nums)]} python eval.py --model_name {model_name} --data_name {data_name} --num_example {num_example} --num_shot {num_shot} --max_token {max_token} --eval_num_shot {eval_num_shot} --activation_path {f'./site/{model_name.split('/')[-1]}/mean_activations'} --alphas_path {f'./site/{model_name.split('/')[-1]}/alphas/alphas_{exp_name}.pt'} --result_folder {f'./site/{model_name.split('/')[-1]}/result'} --cur_mode {cur_mode} --experiment_name {exp_name} --batch_size {batch_size} --test_batch_size {test_batch_size} --data_path {data_path} --dataset_names {' '.join(dataset_names[i] for i in range(len(dataset_names)))} --lr {lr} --epoch {epoch}&'+'\n')    
            cnt += 2
        else:
            if exact_match:
                f.write(f'CUDA_VISIBLE_DEVICES={gpu_nums[cnt % len(gpu_nums)]} python eval.py --exact_match --model_name {model_name} --data_name {data_name} --num_example {num_example} --num_shot {num_shot} --max_token {max_token} --eval_num_shot {eval_num_shot} --activation_path {f'./site/{model_name.split('/')[-1]}/mean_activations'} --alphas_path {f'./site/{model_name.split('/')[-1]}/alphas/alphas_{exp_name}.pt'} --result_folder {f'./site/{model_name.split('/')[-1]}/result'} --cur_mode {cur_mode} --experiment_name {exp_name} --batch_size {batch_size} --test_batch_size {test_batch_size} --data_path {data_path} --dataset_names {' '.join(dataset_names[i] for i in range(len(dataset_names)))} --lr {lr} --epoch {epoch}&'+'\n')    
            else:
                f.write(f'CUDA_VISIBLE_DEVICES={gpu_nums[cnt % len(gpu_nums)]} python eval.py --model_name {model_name} --data_name {data_name} --num_example {num_example} --num_shot {num_shot} --max_token {max_token} --eval_num_shot {eval_num_shot} --activation_path {f'./site/{model_name.split('/')[-1]}/mean_activations'} --alphas_path {f'./site/{model_name.split('/')[-1]}/alphas/alphas_{exp_name}.pt'} --result_folder {f'./site/{model_name.split('/')[-1]}/result'} --cur_mode {cur_mode} --experiment_name {exp_name} --batch_size {batch_size} --test_batch_size {test_batch_size} --data_path {data_path} --dataset_names {' '.join(dataset_names[i] for i in range(len(dataset_names)))} --lr {lr} --epoch {epoch}&'+'\n')    
            cnt += 1
    f.write("\n"+"wait"+"\n")
    f.write("echo 'Successfully ran the code'"+'\n')

In [1]:
# This is for evaluating task-specific soft head-selection parameters (default)
import os
dataset_dirs = ["../dataset"]
dataset_names = []
for dataset_dir in dataset_dirs:
    dataset_names.extend([filename for filename in os.listdir(dataset_dir)])
dataset_names_list = [[dataset_name] for dataset_name in dataset_names]
print(len(dataset_names_list))
dataset_names_list

57


[['adjective_v_verb_3'],
 ['adjective_v_verb_5'],
 ['ag_news'],
 ['alphabetically_first_3'],
 ['alphabetically_first_5'],
 ['alphabetically_last_3'],
 ['alphabetically_last_5'],
 ['animal_v_object_3'],
 ['animal_v_object_5'],
 ['antonym'],
 ['capitalize'],
 ['capitalize_first_letter'],
 ['capitalize_last_letter'],
 ['capitalize_second_letter'],
 ['choose_first_of_3'],
 ['choose_first_of_5'],
 ['choose_last_of_3'],
 ['choose_last_of_5'],
 ['choose_middle_of_3'],
 ['choose_middle_of_5'],
 ['color_v_animal_3'],
 ['color_v_animal_5'],
 ['commonsense_qa'],
 ['concept_v_object_3'],
 ['concept_v_object_5'],
 ['conll2003_location'],
 ['conll2003_organization'],
 ['conll2003_person'],
 ['country-capital'],
 ['country-currency'],
 ['english-french'],
 ['english-german'],
 ['english-spanish'],
 ['fruit_v_animal_3'],
 ['fruit_v_animal_5'],
 ['landmark-country'],
 ['lowercase_first_letter'],
 ['lowercase_last_letter'],
 ['national_parks'],
 ['next_capital_letter'],
 ['next_item'],
 ['object_v_conce

In [ ]:
# This is for evaluating task-agnotic soft head-selection parameters
import os
dataset_dirs = ["../dataset"]
dataset_names = []
for dataset_dir in dataset_dirs:
    dataset_names.extend([filename for filename in os.listdir(dataset_dir)])
dataset_names_list = [dataset_names]

[['adjective_v_verb_3',
  'adjective_v_verb_5',
  'ag_news',
  'alphabetically_first_3',
  'alphabetically_first_5',
  'alphabetically_last_3',
  'alphabetically_last_5',
  'animal_v_object_3',
  'animal_v_object_5',
  'antonym',
  'capitalize',
  'capitalize_first_letter',
  'capitalize_last_letter',
  'capitalize_second_letter',
  'choose_first_of_3',
  'choose_first_of_5',
  'choose_last_of_3',
  'choose_last_of_5',
  'choose_middle_of_3',
  'choose_middle_of_5',
  'color_v_animal_3',
  'color_v_animal_5',
  'commonsense_qa',
  'concept_v_object_3',
  'concept_v_object_5',
  'conll2003_location',
  'conll2003_organization',
  'conll2003_person',
  'country-capital',
  'country-currency',
  'english-french',
  'english-german',
  'english-spanish',
  'fruit_v_animal_3',
  'fruit_v_animal_5',
  'landmark-country',
  'lowercase_first_letter',
  'lowercase_last_letter',
  'national_parks',
  'next_capital_letter',
  'next_item',
  'object_v_concept_3',
  'object_v_concept_5',
  'park-co